In [2]:
import pandas as pd
import datetime
import requests

In [16]:
print(custom)

{'date': '2019-05-30', 'hourly': [{'time': '0', 'tempC': '15', 'tempF': '60', 'windspeedMiles': '4', 'windspeedKmph': '6', 'winddirDegree': '288', 'winddir16Point': 'WNW', 'weatherCode': '119', 'weatherIconUrl': [{'value': 'http://cdn.worldweatheronline.com/images/wsymbols01_png_64/wsymbol_0003_white_cloud.png'}], 'weatherDesc': [{'value': 'Cloudy'}], 'precipMM': '0.1', 'precipInches': '0.0', 'humidity': '80', 'visibility': '10', 'visibilityMiles': '6', 'pressure': '1007', 'pressureInches': '30', 'cloudcover': '71', 'HeatIndexC': '15', 'HeatIndexF': '60', 'DewPointC': '12', 'DewPointF': '54', 'WindChillC': '15', 'WindChillF': '60', 'WindGustMiles': '5', 'WindGustKmph': '9', 'FeelsLikeC': '15', 'FeelsLikeF': '60', 'uvIndex': '1'}, {'time': '100', 'tempC': '15', 'tempF': '59', 'windspeedMiles': '5', 'windspeedKmph': '8', 'winddirDegree': '290', 'winddir16Point': 'WNW', 'weatherCode': '119', 'weatherIconUrl': [{'value': 'http://cdn.worldweatheronline.com/images/wsymbols01_png_64/wsymbol_0

In [3]:
start = datetime.datetime.strptime("01-05-2015", "%d-%m-%Y")
end = datetime.datetime.strptime("01-11-2019", "%d-%m-%Y")
date_generated = [start + datetime.timedelta(days=x) for x in range(0, (end-start).days)]

array = []

for date in date_generated:
    #print (date.strftime("%Y-%m-%d"))
    URL = 'https://api.worldweatheronline.com/premium/v1/past-weather.ashx?key=18713d003c6f45d6870185944210109&q=New%20York&format=json&date='+date.strftime("%Y-%m-%d")+'&tp=24'
    wetter = requests.get('https://api.worldweatheronline.com/premium/v1/past-weather.ashx?key=18713d003c6f45d6870185944210109&q=New York&format=json&date='+date.strftime("%Y-%m-%d")+'&tp=1')
    custom = {"date":wetter.json()["data"]["weather"][0]["date"] } 
    custom.update({"hourly":wetter.json()["data"]["weather"][0]["hourly"]})
    #custom.update({"sunHour" : wetter.json()["data"]["weather"][0]["sunHour"]})
    df = pd.json_normalize(custom,record_path=["hourly"],meta=["date"])
    df = pd.DataFrame(df,columns=["date","time","tempC","precipMM","humidity",'windspeedKmph',''])
    array.append(df)

KeyboardInterrupt: 

In [3]:
wetter = requests.get('https://api.worldweatheronline.com/premium/v1/past-weather.ashx?key=18713d003c6f45d6870185944210109&q=New York&format=json&date='+date.strftime("%Y-%m-%d")+'&tp=1')
print(wetter.json())

{'data': {'request': [{'type': 'City', 'query': 'New York, United States of America'}], 'weather': [{'date': '2019-05-30', 'astronomy': [{'sunrise': '05:28 AM', 'sunset': '08:19 PM', 'moonrise': '03:35 AM', 'moonset': '04:23 PM', 'moon_phase': 'Waning Crescent', 'moon_illumination': '16'}], 'maxtempC': '24', 'maxtempF': '74', 'mintempC': '15', 'mintempF': '59', 'avgtempC': '20', 'avgtempF': '69', 'totalSnow_cm': '0.0', 'sunHour': '12.4', 'uvIndex': '5', 'hourly': [{'time': '0', 'tempC': '15', 'tempF': '60', 'windspeedMiles': '4', 'windspeedKmph': '6', 'winddirDegree': '288', 'winddir16Point': 'WNW', 'weatherCode': '119', 'weatherIconUrl': [{'value': 'http://cdn.worldweatheronline.com/images/wsymbols01_png_64/wsymbol_0003_white_cloud.png'}], 'weatherDesc': [{'value': 'Cloudy'}], 'precipMM': '0.1', 'precipInches': '0.0', 'humidity': '80', 'visibility': '10', 'visibilityMiles': '6', 'pressure': '1007', 'pressureInches': '30', 'cloudcover': '71', 'HeatIndexC': '15', 'HeatIndexF': '60', 'De

In [3]:
new_weather = pd.concat(array,axis=0,ignore_index = True)

In [4]:
print(new_weather)

             date  time tempC precipMM humidity
0      2017-01-01     0     4      0.6       75
1      2017-01-01   100     4      0.3       76
2      2017-01-01   200     4      0.0       78
3      2017-01-01   300     4      0.0       80
4      2017-01-01   400     3      0.0       81
...           ...   ...   ...      ...      ...
17491  2018-12-30  1900     5      0.0       71
17492  2018-12-30  2000     6      0.0       73
17493  2018-12-30  2100     6      0.0       74
17494  2018-12-30  2200     6      0.0       74
17495  2018-12-30  2300     5      0.0       74

[17496 rows x 5 columns]


In [5]:
new_weather.dtypes

date        object
time        object
tempC       object
precipMM    object
humidity    object
dtype: object

In [6]:
new_weather['time'] = new_weather['time'].astype(int)

In [7]:
new_weather['time'] = new_weather['time'] / 100

In [8]:
new_weather['time']

0         0.0
1         1.0
2         2.0
3         3.0
4         4.0
         ... 
17491    19.0
17492    20.0
17493    21.0
17494    22.0
17495    23.0
Name: time, Length: 17496, dtype: float64

In [9]:
new_weather.to_csv('weather082018.csv',index=False)